In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from tqdm import tqdm

from map_boxes import mean_average_precision_for_boxes

In [2]:
def mAP(coco_path):
     
    coco = COCO(coco_path)
    print()
    print(coco_path)

    check_point = '../detection/baseline/faster_rcnn/checkpoints/faster_rcnn_torchvision_checkpoints.pth'
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # torchvision model 불러오기
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 11  # 10 class + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.to(device)
    model.load_state_dict(torch.load(check_point))
    model.eval()

    gt = []
    pred = []

    for image_id in tqdm(coco.getImgIds()):
        
        image_info = coco.loadImgs(image_id)[0]
        annotation_id = coco.getAnnIds(imgIds=image_info['id'])
        annotation_info_list = coco.loadAnns(annotation_id)
        
        file_name = image_info['file_name']
        
        # ground truth
        for annotation in annotation_info_list:
            gt.append([file_name, annotation['category_id'],
                       float(annotation['bbox'][0]),
                       float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                       float(annotation['bbox'][1]),
                       (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

        # pred
        image = cv2.imread(os.path.join('../detection/dataset/', file_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)
        image = image.to(device)

        outputs = model(image.reshape(1, 3, 1024, 1024))

        for output in outputs:
            for box, score, label in zip(output['boxes'].tolist(), output['scores'].tolist(), output['labels'].tolist()):
                if score > 0.05:
                    pred.append([file_name, label-1, float(score), float(box[0]), float(box[2]), float(box[1]), float(box[3])])
    
    
    mean_ap, average_precisions = mean_average_precision_for_boxes(gt, pred, iou_threshold=0.5)
    print()


In [3]:
for data in os.listdir('../detection/dataset'):
    if '.json' in data and 'test' not in data:
        mAP(f'../detection/dataset/{data}')

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
../detection/dataset/train.json


100%|██████████| 4883/4883 [05:53<00:00, 13.81it/s]


Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.518705 |    3966
1                              | 0.657203 |    6352
2                              | 0.700753 |     897
3                              | 0.654967 |     936
4                              | 0.787895 |     982
5                              | 0.614069 |    2943
6                              | 0.642534 |    1263
7                              | 0.818377 |    5178
8                              | 0.820724 |     159
9                              | 0.747097 |     468
mAP: 0.696232

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
../detection/dataset/cv5_train3.json


100%|██████████| 3902/3902 [04:50<00:00, 13.45it/s]


Number of files in annotations: 3902
Number of files in predictions: 3902
Unique classes: 10
Detections length: 3902
Annotations length: 3902
0                              | 0.524958 |    3192
1                              | 0.653932 |    5069
2                              | 0.690553 |     715
3                              | 0.672414 |     749
4                              | 0.780809 |     779
5                              | 0.616092 |    2330
6                              | 0.647207 |    1013
7                              | 0.819161 |    4132
8                              | 0.835584 |     116
9                              | 0.739547 |     359
mAP: 0.698026

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
../detection/dataset/cv5_val4.json


100%|██████████| 974/974 [01:13<00:00, 13.34it/s]


Number of files in annotations: 974
Number of files in predictions: 974
Unique classes: 10
Detections length: 974
Annotations length: 974
0                              | 0.532272 |     778
1                              | 0.663924 |    1155
2                              | 0.759982 |     168
3                              | 0.674855 |     191
4                              | 0.825498 |     211
5                              | 0.605977 |     545
6                              | 0.686436 |     181
7                              | 0.817350 |    1008
8                              | 0.895542 |      32
9                              | 0.702017 |      92
mAP: 0.716385

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
../detection/dataset/cv5_train4.json


 12%|█▏        | 476/3909 [00:35<04:16, 13.39it/s]


KeyboardInterrupt: 